# CoinGecko Categories Scraper v15

In [ ]:
!python -m pip install -q pycoingecko aiohttp aiodns pandas tqdm pyarrow nest_asyncio

In [ ]:
import asyncio, aiohttp, nest_asyncio, pandas as pd, time, tqdm
nest_asyncio.apply()

VS_CCY='usd'
CONCURRENCY=12
PAGES_PER_CAT=12

SLUG_MAP={
    'ai':['ai-big-data','artificial-intelligence'],
    'gaming':['gaming','metaverse'],
    'rwa':['real-world-assets','rwa-protocol','tokenized-commodities'],
    'meme':['memes'],
}

async def fetch_page(sess, slug, page, retries=3):
    url='https://api.coingecko.com/api/v3/coins/markets'
    params=dict(vs_currency=VS_CCY, category=slug, per_page=250, page=page, sparkline='false')
    for att in range(retries):
        async with sess.get(url, params=params, timeout=20) as r:
            if r.status==429:
                await asyncio.sleep(1.2+att); continue
            if r.status!=200:
                return []
            try:
                data=await r.json()
                return data if isinstance(data, list) else []
            except aiohttp.ContentTypeError:
                await asyncio.sleep(1+att)
    return []

async def gather(slug_map):
    sem=asyncio.Semaphore(CONCURRENCY)
    rows=[]
    async with aiohttp.ClientSession() as sess:
        async def worker(nar, slug, pg):
            async with sem:
                data=await fetch_page(sess, slug, pg)
                for d in data:
                    d['narrative']=nar
                rows.extend(data)
        tasks=[worker(nar, slug, pg) for nar, slugs in slug_map.items() for slug in slugs for pg in range(1, PAGES_PER_CAT+1)]
        for t in tqdm.tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            await t
    return rows

t0=time.time()
rows=asyncio.get_event_loop().run_until_complete(gather(SLUG_MAP))
print('Tiempo:', round(time.time()-t0,1),'s, filas brutas', len(rows))

df=(pd.DataFrame(rows)
    .rename(columns={'current_price':'price','total_volume':'volume'})
    [['id','symbol','name','narrative','price','volume','market_cap']]
    .drop_duplicates('id'))
print(df['narrative'].value_counts())
print('Total únicos:', len(df))
df.to_csv('cryptos_filtered.csv', index=False)
print('Guardado cryptos_filtered.csv')
